# Preview Evaluation - Perplexity

Thia is a notebook to make a preview comparison of the generations when using LLM JUDGE

In [1]:
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
from utils.metrics.calculate_metric import calculate_agg_metric
import os
import numpy as np
import json

/home/caio.rhoden/miniconda3/envs/nq_dl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Calculate Metrics and save them

In [4]:
rag_metrics = ["ip", "l2", "cosine"]
path  = "experiments"

In [5]:
for s in rag_metrics:
    questions_path = "../questions_250_42_dev.feather"
    for file in os.listdir(f"../{path}_{s}/generations"):
        print(file)
        calculate_agg_metric(
            metrics=["rouge_l"],
            generation_path=f"../{path}_{s}/generations/{file}",
            reference_path=questions_path,
            saving_path=f"result_{s}_{file.split('.')[0]}.feather"
        )




rag_generations.json
rag_generations.json
rag_generations.json


## Load Data

In [7]:
### Perplexity results

_perplexitiy_results = []
for f in os.listdir("."):
    if f.endswith("feather"):
        processsed_namefile = f.split(".")[0].split("result_")[1]
        run_type = processsed_namefile.split("_")[0]
        _perplexitiy_results.append(pl.read_ipc(f).with_columns( pl.lit(run_type).alias("run_type")))
perplexity_results = pl.concat(_perplexitiy_results)
perplexity_results.group_by("run_type", "metric").agg(pl.col("mean").mean()).sort("run_type")

Could not memory_map compressed IPC file, defaulting to normal read. Toggle off 'memory_map' to silence this warning.
Could not memory_map compressed IPC file, defaulting to normal read. Toggle off 'memory_map' to silence this warning.
Could not memory_map compressed IPC file, defaulting to normal read. Toggle off 'memory_map' to silence this warning.


run_type,metric,mean
str,str,f64
"""cosine""","""rouge_l""",0.139944
"""ip""","""rouge_l""",0.119726
"""l2""","""rouge_l""",0.10464


In [77]:
import json
indexes = {}
for s in rag_metrics:
    rag_retrieval = json.load(open(f"../{path}_{s}/retrieval/rag_retrieval_indexes.json"))

    # Get values in order and convert to numpy arrayzz
    array_250x100 = np.array([rag_retrieval[str(key)] for key in range(250)])
    # Take first 16 elements
    array_250x16 = array_250x100[:, 84:100]
    # Convert back to list if needed (otherwise keep as numpy array)

    indexes[str(s)] = array_250x16 

In [78]:
len(indexes["ip"])

250

In [81]:


def count_matches_no_numpy(list_a, list_b):
    matches = 0
    for key in range(250):
        for i in range(16):

            if list_a[key][i] in list_b[key]:
                
                matches += 1
    return matches

def count_matches_no_numpy_3(list_a, list_b, list_c):
    matches = 0
    for key in range(250):
        for i in range(16):
            if list_a[key][i] in list_b[key] and list_a[key][i] in list_c[key]:
                matches += 1
    return matches

print(f"Intersection between IP and L2: {count_matches_no_numpy(indexes['ip'], indexes['l2'])}")
print(f"Intersection between IP and Cosine: {count_matches_no_numpy(indexes['ip'], indexes['cosine'])}")
print(f"Intersection between Cosine and L2: {count_matches_no_numpy(indexes['cosine'], indexes['l2'])}")
print(f"Intersection between IP, L2 and LLM Cosine: {count_matches_no_numpy_3(indexes['ip'], indexes['l2'], indexes['cosine'])}")

Intersection between IP and L2: 1106
Intersection between IP and Cosine: 3543
Intersection between Cosine and L2: 1124
Intersection between IP, L2 and LLM Cosine: 1045


In [82]:
print(f"Max intersection = {16*250}")

Max intersection = 4000
